# Testing

## Imports

In [956]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Functions

In [957]:
def generate_random_rank_r_matrix(m):
    n = int(0.5 * m)
    r = int(np.floor(0.25 * m))
    while True:
        A = np.random.rand(m, n)
        U, S, VT = np.linalg.svd(A)
        S_bar = np.zeros((m, n))
        if S.shape[0] < r:
            continue
        for i in range(r):
            S_bar[i, i] = S[i]
        A = np.dot(U, np.dot(S_bar, VT))
        return A

In [958]:
def matrix_rank(H):
    U, S, VT = np.linalg.svd(H)
    rank = 0
    for i in range(S.shape[0]):
        if S[i] > epsilon:
            rank += 1
    return rank

In [959]:
def matrix_vec_inf_norm(H):
    return max([np.abs(h) for h in H.flatten()])

In [960]:
def matrix_frobenius_norm(H):
    return np.linalg.norm(H, ord="fro")

In [961]:
def initial_variables(V1, U1, D_inv, rho):
    Theta = np.dot(V1, U1.T) / matrix_vec_inf_norm(np.dot(V1, U1.T))
    Lambda = Theta / rho
    E = np.dot(V1, np.dot(D_inv, U1.T)) + Lambda
    return Lambda, E

In [962]:
def soft_thresholding(a, kappa):
    if a > kappa:
        return a - kappa
    elif np.abs(a) <= kappa:
        return 0
    elif a < -kappa:
        return a + kappa

In [964]:
def admm1_123(A, rho, epsilon_abs, epsilon_rel):
    # Gets dimensions of matrix A
    m = A.shape[0]
    n = A.shape[1]
    # Calculates full singular value decomposition of A
    U, S, VT = np.linalg.svd(A)
    # Calculates rank of A
    r = matrix_rank(A)
    # Calculates variables U1, V1, V2 and D^{-1}
    U1 = U[:, :r]
    V1 = VT.T[:, :r]
    V2 = VT.T[:, r:]
    D_inv = np.zeros((r, r))
    for i in range(r):
            D_inv[i, i] = 1 / S[i]
    # Calculates initial variables
    Lambda, Ekm = initial_variables(V1=V1, U1=U1, D_inv=D_inv, rho=rho)
    Ek = np.zeros((n, m))
    while True:
        # Updates variable J
        J = Ekm - np.dot(V1, np.dot(D_inv, U1.T)) - Lambda
        # Updates variable Zk
        Z = np.dot(V2.T, np.dot(J, U1))
        # Updates variable Y
        Y = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T)) + Lambda
        # Updates variable Ek
        for i in range(n):
            for j in range(m):
                Ek[i, j] = soft_thresholding(a=Y[i, j], kappa=1/rho)
        # Updates variable Lambdak
        Lambda = Lambda + np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T)) - Ek
        # Calculates stop criterion variables
        rk = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T)) - Ek
        sk = rho * np.dot(V2.T, np.dot(Ek - Ekm, U1))
        matrix_norms = [
            matrix_frobenius_norm(Ek),
            matrix_frobenius_norm(np.dot(V2, np.dot(Z, U1.T))),
            matrix_frobenius_norm(np.dot(V1, np.dot(D_inv, U1.T)))
        ]
        primal_upper_bound = epsilon_abs * np.sqrt(m*n) + epsilon_rel * max(matrix_norms)
        aux_var = matrix_frobenius_norm(np.dot(V2.T, np.dot(Lambda, U1)))
        dual_upper_bound = epsilon_abs * np.sqrt((n-r)*r) + epsilon_rel * rho * aux_var
        # Checks stop criterion
        if (matrix_frobenius_norm(rk) <= primal_upper_bound) and (matrix_frobenius_norm(sk)) < dual_upper_bound:
            break
        # Makes Ek the new Ek-1
        Ekm = Ek
    # Calculates output matrix H
    H = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(Z, U1.T))
    return H

In [965]:
def admm1_134(A, rho, epsilon_abs, epsilon_rel):
    # Gets dimensions of matrix A
    m = A.shape[0]
    n = A.shape[1]
    # Calculates full singular value decomposition of A
    U, S, VT = np.linalg.svd(A)
    # Calculates rank of A
    r = matrix_rank(A)
    # Calculates variables U1, U2, V1, V2 and D^{-1}
    U1 = U[:, :r]
    U2 = U[:, r:]
    V1 = VT.T[:, :r]
    V2 = VT.T[:, r:]
    D_inv = np.zeros((r, r))
    for i in range(r):
            D_inv[i, i] = 1 / S[i]
    # Calculates initial variables
    Lambda, Ekm = initial_variables(V1=V1, U1=U1, D_inv=D_inv, rho=rho)
    Ek = np.zeros((n, m))
    while True:
        # Updates variable J
        J = Ekm - np.dot(V1, np.dot(D_inv, U1.T)) - Lambda
        # Updates variable Wk
        W = np.dot(V2.T, np.dot(J, U2))
        # Updates variable Y
        Y = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T)) + Lambda
        # Updates variable Ek
        for i in range(n):
            for j in range(m):
                Ek[i, j] = soft_thresholding(a=Y[i, j], kappa=1/rho)
        # Updates variable Lambdak
        Lambda = Lambda + np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T)) - Ek
        # Calculates stop criterion variables
        rk = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T)) - Ek
        sk = rho * np.dot(V2.T, np.dot(Ek - Ekm, U1))
        matrix_norms = [
            matrix_frobenius_norm(Ek),
            matrix_frobenius_norm(np.dot(V2, np.dot(W, U2.T))),
            matrix_frobenius_norm(np.dot(V1, np.dot(D_inv, U1.T)))
        ]
        primal_upper_bound = epsilon_abs * np.sqrt(m*n) + epsilon_rel * max(matrix_norms)
        aux_var = matrix_frobenius_norm(np.dot(V2.T, np.dot(Lambda, U1)))
        dual_upper_bound = epsilon_abs * np.sqrt((n-r)*r) + epsilon_rel * rho * aux_var
        # Checks stop criterion
        if (matrix_frobenius_norm(rk) <= primal_upper_bound) and (matrix_frobenius_norm(sk)) < dual_upper_bound:
            break
        # Makes Ek the new Ek-1
        Ekm = Ek
    # Calculates output matrix H
    H = np.dot(V1, np.dot(D_inv, U1.T)) + np.dot(V2, np.dot(W, U2.T))
    return H

## Testing Functions

In [966]:
epsilon = 10 ** -5

m = 10
A = generate_random_rank_r_matrix(m=m)
rho = 0.5
epsilon_abs = 10 ** -5
epsilon_rel = 10 ** -6

In [967]:
H = admm1_123(A=A, rho=rho, epsilon_abs=epsilon_abs, epsilon_rel=epsilon_rel)

AHA = np.dot(A, np.dot(H, A))
HAH = np.dot(H, np.dot(A, H))
AH = np.dot(A, H)
AH_T = np.dot(A, H).T
HA = np.dot(H, A)
HA_T = np.dot(H, A).T
print(matrix_frobenius_norm(AHA - A))
print(matrix_frobenius_norm(HAH - H))
print(matrix_frobenius_norm(AH - AH_T))
print(matrix_frobenius_norm(HA - HA_T))

1.4574981058852602e-15
3.3081405866629856e-16
4.748961954867035e-16
2.0502246011559477


In [968]:
H = admm1_134(A=A, rho=rho, epsilon_abs=epsilon_abs, epsilon_rel=epsilon_rel)

AHA = np.dot(A, np.dot(H, A))
HAH = np.dot(H, np.dot(A, H))
AH = np.dot(A, H)
AH_T = np.dot(A, H).T
HA = np.dot(H, A)
HA_T = np.dot(H, A).T
print(matrix_frobenius_norm(AHA - A))
print(matrix_frobenius_norm(HAH - H))
print(matrix_frobenius_norm(AH - AH_T))
print(matrix_frobenius_norm(HA - HA_T))

6.986263637713189e-16
0.5564169003701275
7.409686085767678e-16
5.171222029444952e-16
